# Spark SQL and DataFrames

### 1. Basic manipulations
First, let's create some users:

In [0]:
from datetime import date

# Let's first create some users:
col_names = ["first_name", "last_name", "birth_date", "gender", "country"]
users = [
  ("Alice", "Jones", date(1981, 4, 15), "female", "Canada"),
  ("John", "Doe", date(1951, 1, 21), "male", "USA"),
  ("Barbara", "May", date(1951, 9, 1), "female", "Australia"),
  ("James", "Smith", date(1975, 7, 12), "male", "United Kingdom"),
  ("Gerrard", "Dupont", date(1968, 5, 9), "male", "France"),
  ("Amanda", "B.", date(1988, 12, 16), "female", "New Zeland")
]

users_df = spark.createDataFrame(users, col_names)
display(users_df) # Only works in Databricks. Elswehere, use "df.show()" or "df.toPandas()"

first_name,last_name,birth_date,gender,country
Alice,Jones,1981-04-15,female,Canada
John,Doe,1951-01-21,male,USA
Barbara,May,1951-09-01,female,Australia
James,Smith,1975-07-12,male,United Kingdom
Gerrard,Dupont,1968-05-09,male,France
Amanda,B.,1988-12-16,female,New Zeland


Now it's your turn, create more users (at least 3, with different names) and add to the initial users, saving the result in a new variable.

In [0]:
new_users_df = spark.createDataFrame([("Marie", "Dubois", date(1990, 3, 15),"male", "France"),
                                      ("Lucas", "Veitman", date(1950, 1, 21), "male", "USA"),
                                      ("Bob", "Tirole", date(1951, 8, 10), "male", "Australia")],col_names)
all_users_df = users_df.union(new_users_df)
display(all_users_df) # or all_users_df.show()

first_name,last_name,birth_date,gender,country
Alice,Jones,1981-04-15,female,Canada
John,Doe,1951-01-21,male,USA
Barbara,May,1951-09-01,female,Australia
James,Smith,1975-07-12,male,United Kingdom
Gerrard,Dupont,1968-05-09,male,France
Amanda,B.,1988-12-16,female,New Zeland
Marie,Dubois,1990-03-15,male,France
Lucas,Veitman,1950-01-21,male,USA
Bob,Tirole,1951-08-10,male,Australia


Now, select only two columns and show the resulting DataFrame, without saving it into a variable.

In [0]:
all_users_df.select("first_name","country").show()

+----------+--------------+
|first_name|       country|
+----------+--------------+
|     Alice|        Canada|
|      John|           USA|
|   Barbara|     Australia|
|     James|United Kingdom|
|   Gerrard|        France|
|    Amanda|    New Zeland|
|     Marie|        France|
|     Lucas|           USA|
|       Bob|     Australia|
+----------+--------------+



Now, register your DataFrame as a table and select the same two columns with a SQL query string

In [0]:
query_string = "select first_name, country from new_view"
all_users_df.createOrReplaceTempView("new_view") # creates a local temporary table accessible by a SQL query
spark.sql(query_string).show()

+----------+--------------+
|first_name|       country|
+----------+--------------+
|     Alice|        Canada|
|      John|           USA|
|   Barbara|     Australia|
|     James|United Kingdom|
|   Gerrard|        France|
|    Amanda|    New Zeland|
|     Marie|        France|
|     Lucas|           USA|
|       Bob|     Australia|
+----------+--------------+



Now we want to add an unique identifier for each user in the table. There are many strategies for that, and for our example we will use the string `{last_name}_{first_name}`

You can use `all_users_df` since your latest operation did not override its values. Add a new column called `user_id` to your DataFrame and save to a new variable.

**Hint:** The first place to look is in the [functions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) package

In [0]:
from pyspark.sql import functions as fn

user_id = fn.concat_ws(" ", all_users_df.last_name, all_users_df.first_name) 

users_with_id = all_users_df.withColumn("user_id", user_id)
display(users_with_id)

first_name,last_name,birth_date,gender,country,user_id
Alice,Jones,1981-04-15,female,Canada,Jones Alice
John,Doe,1951-01-21,male,USA,Doe John
Barbara,May,1951-09-01,female,Australia,May Barbara
James,Smith,1975-07-12,male,United Kingdom,Smith James
Gerrard,Dupont,1968-05-09,male,France,Dupont Gerrard
Amanda,B.,1988-12-16,female,New Zeland,B. Amanda
Marie,Dubois,1990-03-15,male,France,Dubois Marie
Lucas,Veitman,1950-01-21,male,USA,Veitman Lucas
Bob,Tirole,1951-08-10,male,Australia,Tirole Bob


You can also do the same thing with an User Defined Function by passing a lambda, although it is not recommended when there is already a function in the `functions` package.

Add a new column called `user_id_udf` to the DataFrame, using an UDF that receives two parameters and concatenate them.

In [0]:
concat_udf = fn.udf(lambda x, y: x + " " + y)
users_with_id_udf = all_users_df.withColumn("user_id_udf", concat_udf(all_users_df["last_name"], all_users_df["first_name"]))
display(users_with_id_udf)

first_name,last_name,birth_date,gender,country,user_id_udf
Alice,Jones,1981-04-15,female,Canada,Jones Alice
John,Doe,1951-01-21,male,USA,Doe John
Barbara,May,1951-09-01,female,Australia,May Barbara
James,Smith,1975-07-12,male,United Kingdom,Smith James
Gerrard,Dupont,1968-05-09,male,France,Dupont Gerrard
Amanda,B.,1988-12-16,female,New Zeland,B. Amanda
Marie,Dubois,1990-03-15,male,France,Dubois Marie
Lucas,Veitman,1950-01-21,male,USA,Veitman Lucas
Bob,Tirole,1951-08-10,male,Australia,Tirole Bob


Now, let's add another column called `age` with the computed age (in years) of each user, based on a given reference date, and save the resulting DataFrame into a new variable.

**Hint:** You can first compute the age in months, and then divide by 12. A final operation will probably be needed to get an integer number.

In [0]:
from pyspark.sql.functions import *

reference_date = date(2017, 12, 31)
users_with_age = users_with_id.withColumn("age",round(months_between(lit(reference_date), users_with_id.birth_date)/lit(12),0))
display(users_with_age)

first_name,last_name,birth_date,gender,country,user_id,age
Alice,Jones,1981-04-15,female,Canada,Jones Alice,37.0
John,Doe,1951-01-21,male,USA,Doe John,67.0
Barbara,May,1951-09-01,female,Australia,May Barbara,66.0
James,Smith,1975-07-12,male,United Kingdom,Smith James,42.0
Gerrard,Dupont,1968-05-09,male,France,Dupont Gerrard,50.0
Amanda,B.,1988-12-16,female,New Zeland,B. Amanda,29.0
Marie,Dubois,1990-03-15,male,France,Dubois Marie,28.0
Lucas,Veitman,1950-01-21,male,USA,Veitman Lucas,68.0
Bob,Tirole,1951-08-10,male,Australia,Tirole Bob,66.0


Now, an analytical question: How many users of each gender who are more than 40 years old exist in this data? The solution must be a DataFrame with two columns: `age` and `count` and two lines, one for each gender.

Bonus: Try to do your solution in a single chain (without intermediate variables)

**Hint:** You will need to filter and aggregate the data.

In [0]:
result = users_with_age.groupBy(users_with_age.gender).agg(count(when(users_with_age.age > 40, 1)).alias("count"))
display(result)

gender,count
female,1
male,5


### 2. Reading files, performing joins, and aggregating

For this section you will use some fake data of two datasets: `Users` and `Donations`. The data is provided in two CSV files *with header* and using *comma* as separator.

The `Users` dataset contains information about about the users. 

The `Donations` dataset contains information about Donations performed by those users.

The first task is to read these files into the appropriate DataFrames.

**Note:** You need to set the option "inferSchema" to true in order to have the columns in the correct types.

_The data for this section has been created using [Mockaroo](https://www.mockaroo.com/)_.

In [0]:
users_from_file = spark.read.csv("/FileStore/tables/users-1.csv", sep = ",", header = True, inferSchema = True)
donations = spark.read.csv("/FileStore/tables/donations-1.csv", sep = ",", header = True, inferSchema = True)

Now investigate the columns, contents and size of both datasets

In [0]:
# print the column names and types
users_from_file.printSchema()
donations.printSchema()

# print 5 elements of the datasets in a tabular format
users_from_file.limit(5).show()
donations.limit(5).show()

# print the number of lines of each dataset
print (users_from_file.count())
print (donations.count())

root
 |-- user_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- country: string (nullable = true)

root
 |-- donation_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- donation_value: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- timestamp: string (nullable = true)

+-------+----------+---------+------+----------+-------------+
|user_id|first_name|last_name|gender|birth_date|      country|
+-------+----------+---------+------+----------+-------------+
|      1|   Janifer|   Lagadu|Female|1940-08-09|United States|
|      2|     Bruis|  Danilov|  Male|1966-05-28|       Brazil|
|      3|      Omar| Spinozzi|  Male|1977-08-28|        China|
|      4|       Mag|  Kennedy|Female|1963-11-11|        China|
|      5|    Gerome|   Lushey|  Male|1990-06-08|       Brazil|

**Note:** If all the column types shown in the previous results are "string", you need to make sure you passed "inferSchema" as true when reading the CSV files before continuing.

Before using the data, we may want to add some information about the users. 

Add a column containing the age of each user.

In [0]:
users_from_file = users_from_file.withColumn("age",round(months_between(lit(reference_date), users_from_file.birth_date)/lit(12)))
users_from_file.show(10)

+-------+----------+---------+------+----------+-------------+----+
|user_id|first_name|last_name|gender|birth_date|      country| age|
+-------+----------+---------+------+----------+-------------+----+
|      1|   Janifer|   Lagadu|Female|1940-08-09|United States|77.0|
|      2|     Bruis|  Danilov|  Male|1966-05-28|       Brazil|52.0|
|      3|      Omar| Spinozzi|  Male|1977-08-28|        China|40.0|
|      4|       Mag|  Kennedy|Female|1963-11-11|        China|54.0|
|      5|    Gerome|   Lushey|  Male|1990-06-08|       Brazil|28.0|
|      6|   Lazarus|   Andrus|  Male|1983-04-22|       France|35.0|
|      7|     Adora| Bartunek|Female|1971-06-04|       France|47.0|
|      8|    Gladys|  Gulland|Female|1959-07-07|       Brazil|58.0|
|      9|  Pearline|  Kitcher|Female|1944-06-01|        China|74.0|
|     10|     Jolie|   Diggin|Female|1944-07-31|      Nigeria|73.0|
+-------+----------+---------+------+----------+-------------+----+
only showing top 10 rows



Another useful information to have is the age **range** of each user. Using the `when` function, create the following 5 age ranges:
- "(0, 25]"
- "(25, 35]"
- "(35, 45]"
- "(45, 55]"
- "(55, ~]"

And add a new column to the users DataFrame, containing this information.

**Note:** When building logical operations with Spark DataFrames, it's better to be add parantheses. Example:
```python
df.select("name", "age").where( (df("age") > 20) & (df("age") <= 30) )
```

**Note 2:** If you are having problems with the `when` function, you can make an User Defined Function and do your logic in standard python.

In [0]:
range_of_range = fn.when((users_from_file.age <= 25), '(0,25]')\
                   .when((users_from_file.age > 25) & (users_from_file.age <= 35), '(25,35]')\
                   .when((users_from_file.age > 35) & (users_from_file.age <= 45), '(35,45]')\
                   .when((users_from_file.age > 45) & (users_from_file.age <= 55), '(45,55]')\
                   .otherwise('(55,~]')

users_from_file = users_from_file.withColumn("range of age", range_of_range)
users_from_file.show(10)

+-------+----------+---------+------+----------+-------------+----+------------+
|user_id|first_name|last_name|gender|birth_date|      country| age|range of age|
+-------+----------+---------+------+----------+-------------+----+------------+
|      1|   Janifer|   Lagadu|Female|1940-08-09|United States|77.0|      (55,~]|
|      2|     Bruis|  Danilov|  Male|1966-05-28|       Brazil|52.0|     (45,55]|
|      3|      Omar| Spinozzi|  Male|1977-08-28|        China|40.0|     (35,45]|
|      4|       Mag|  Kennedy|Female|1963-11-11|        China|54.0|     (45,55]|
|      5|    Gerome|   Lushey|  Male|1990-06-08|       Brazil|28.0|     (25,35]|
|      6|   Lazarus|   Andrus|  Male|1983-04-22|       France|35.0|     (25,35]|
|      7|     Adora| Bartunek|Female|1971-06-04|       France|47.0|     (45,55]|
|      8|    Gladys|  Gulland|Female|1959-07-07|       Brazil|58.0|      (55,~]|
|      9|  Pearline|  Kitcher|Female|1944-06-01|        China|74.0|      (55,~]|
|     10|     Jolie|   Diggi

Now that we have improved our users' DataFrame, the first analysis we want to make is the average donation performed by each gender. However, the gender information and the donation value are in different tables, so first we need to join them, using the `user_id` as joining key.

**Note:** Make sure you are not using the `users` DataFrame from the first part of the lab.

**Note 2:** For better performance, you can [broadcast](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.broadcast) the smaller dataset. But you should only do this when you are sure the DataFrame fits in the memory of the nodes.

In [0]:
joined_df = users_from_file.join(donations,'user_id')
display(joined_df.limit(10))

user_id,first_name,last_name,gender,birth_date,country,age,range of age,donation_id,donation_value,date,time,timestamp
63,Tabor,Bramer,Male,1983-05-29,Brazil,35.0,"(25,35]",1,25.41,2017-03-06,18:28:20,2017-03-06 18:28:20
53,Chris,Climie,Male,1970-06-02,France,48.0,"(45,55]",2,93.32,2017-03-15,15:15:09,2017-03-15 15:15:09
74,Livia,Lodemann,Female,1967-03-11,United States,51.0,"(45,55]",3,136.04,2017-10-29,20:36:23,2017-10-29 20:36:23
94,Kendre,Pankhurst.,Female,1941-05-28,China,77.0,"(55,~]",4,37.25,2017-05-16,12:33:58,2017-05-16 12:33:58
83,Quincy,Marquess,Male,1997-08-13,Brazil,20.0,"(0,25]",5,89.45,2017-05-09,22:45:44,2017-05-09 22:45:44
80,Boothe,Endley,Male,1960-03-03,China,58.0,"(55,~]",6,163.67,2017-07-25,21:50:36,2017-07-25 21:50:36
19,Sharl,Airy,Female,1941-09-08,Brazil,76.0,"(55,~]",7,10.69,2017-11-15,8:43:54,2017-11-15 8:43:54
53,Chris,Climie,Male,1970-06-02,France,48.0,"(45,55]",8,135.09,2017-04-14,7:43:37,2017-04-14 7:43:37
46,Ethelyn,Raubenheim,Female,1953-07-19,Brazil,64.0,"(55,~]",9,94.38,2017-03-07,10:30:34,2017-03-07 10:30:34
90,Farlee,O'Connell,Male,1985-05-15,China,33.0,"(25,35]",10,68.63,2017-02-02,14:23:43,2017-02-02 14:23:43


Now, use aggregation to find the the min, max and avg donation by gender.

In [0]:
donations_by_gender = joined_df.groupBy('gender').agg(
                      fn.round(fn.avg('donation_value'), 2).alias('average'),
                      fn.max('donation_value').alias('max'),
                      fn.min('donation_value').alias('min')
)
donations_by_gender.show()

+------+-------+------+----+
|gender|average|   max| min|
+------+-------+------+----+
|Female|  97.63|199.87|1.25|
|  Male| 100.24|199.93|1.12|
+------+-------+------+----+



Now, make the necessary transformations and aggregations to answer to the following questions about the data. Note that some questions are only about the users, so make sure to use the smaller possible dataset when looking for your answers!

**Question 1:** 

a) What's the average, min and max age of the users? 

b) What's the average, min and max age of the users, by gender?

In [0]:
result_1a = users_from_file.agg(fn.round(fn.avg('age'), 2).alias('average'),
                                fn.max('age').alias('max'),
                                fn.min('age').alias('min')
)
print("Average, min, and max age of the users:")
result_1a.show()

result_1b = users_from_file.groupBy('gender').agg(fn.round(fn.avg('age'), 2).alias('average'),
                                                  fn.max('age').alias('max'),
                                                  fn.min('age').alias('min')
)
print("Average, min, and max age of the users by gender:")
result_1b.show()

Average, min, and max age of the users:
+-------+----+----+
|average| max| min|
+-------+----+----+
|  48.68|78.0|19.0|
+-------+----+----+

Average, min, and max age of the users by gender:
+------+-------+----+----+
|gender|average| max| min|
+------+-------+----+----+
|Female|  52.98|78.0|19.0|
|  Male|  44.55|76.0|19.0|
+------+-------+----+----+



**Question 2:**

a) How many distinct country origins exist in the data? Print a DataFrame listing them.

b) What are the top 5 countries with the most users? Print a DataFrame containing the name of the countries and the counts.

In [0]:
result_2a = users_from_file.select("country").distinct()
result_2a.show()

result_2b = users_from_file.groupBy("country").agg(count("user_id").alias("count")).orderBy(desc("count"))
result_2b.show(5)

+-------------+
|      country|
+-------------+
|      Germany|
|       France|
|United States|
|        China|
|      Nigeria|
|       Brazil|
|        Japan|
|  New Zealand|
+-------------+

+-------------+-----+
|      country|count|
+-------------+-----+
|        China|   54|
|       Brazil|   19|
|       France|   13|
|United States|    5|
|        Japan|    4|
+-------------+-----+
only showing top 5 rows



**Question 3:**

What's the number of donations average, min and max donations values by age range?

In [0]:
result_3 = joined_df.groupBy('range of age').agg(
                      fn.round(fn.avg('donation_value'), 2).alias('average'),
                      fn.max('donation_value').alias('max'),
                      fn.min('donation_value').alias('min')
).orderBy('range of age')
result_3.show()

+------------+-------+------+----+
|range of age|average|   max| min|
+------------+-------+------+----+
|      (0,25]|  99.38|199.28|4.33|
|     (25,35]|  95.58|199.87|1.12|
|     (35,45]|  98.21|199.32|2.03|
|     (45,55]| 102.54|199.93|1.19|
|      (55,~]|  99.64|197.79|1.36|
+------------+-------+------+----+



**Question 4:**

a) What's the number of donations, average, min and max donation values by user location (country)?

b) What is the number of donations, average, min and max donation values by gender for each user location (contry)? (the resulting DataFrame must contain 5 columns: the gender of the user, their country and the 3 metrics)

In [0]:
result_4a = joined_df.groupBy('country').agg(
                      count('donation_id').alias('number of donations'),
                      fn.round(fn.avg('donation_value'), 2).alias('average'),
                      fn.min('donation_value').alias('min'),
                      fn.max('donation_value').alias('max')
).orderBy('country')
result_4a.show()

result_4b = joined_df.groupBy('country','gender').agg(
                      count('donation_id').alias('number of donations'),
                      fn.round(fn.avg('donation_value'), 2).alias('average'),
                      fn.min('donation_value').alias('min'),
                      fn.max('donation_value').alias('max')
).orderBy('country')
result_4b.show()

+-------------+-------------------+-------+-----+------+
|      country|number of donations|average|  min|   max|
+-------------+-------------------+-------+-----+------+
|       Brazil|                180| 105.71| 1.12|199.91|
|        China|                563|  97.62| 1.25|199.32|
|       France|                120|  95.63| 5.79|199.93|
|      Germany|                 19|  95.64|17.81|180.48|
|        Japan|                 37|  95.63| 1.19|199.76|
|  New Zealand|                 11| 107.93|12.98|172.51|
|      Nigeria|                 20|  81.23|14.47|192.63|
|United States|                 50| 108.38| 1.36|198.15|
+-------------+-------------------+-------+-----+------+

+-------------+------+-------------------+-------+-----+------+
|      country|gender|number of donations|average|  min|   max|
+-------------+------+-------------------+-------+-----+------+
|       Brazil|  Male|                108| 108.14| 1.12|199.91|
|       Brazil|Female|                 72| 102.05| 1.49|196

**Question 5**

Which month of the year has the largest aggregated donation value?

**Hint**: you can use a function to extract the month from a date, then you can aggregate to find the total donation value.

In [0]:
month = fn.month(joined_df.date).alias('month')
result_5 = joined_df.groupBy(month).agg(fn.round(fn.sum('donation_value'),2).alias('sum')).orderBy(month).show()

+-----+--------+
|month|     sum|
+-----+--------+
|    1| 6752.16|
|    2| 4576.69|
|    3| 7646.93|
|    4| 8308.58|
|    5| 10696.4|
|    6| 8354.05|
|    7| 8151.22|
|    8| 8344.85|
|    9| 8971.71|
|   10| 8287.28|
|   11| 8530.99|
|   12|10430.54|
+-----+--------+



### 3. Window Functions

_This section uses the same data as the last one._

Window functions are very useful for gathering aggregated data without actually aggregating the DataFrame. They can also be used to find "previous" and "next" information for entities, such as an user. [This article](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) has a very nice explanation about the concept.

We want to find the users who donated less than a threshold and remove them from the donations dataset. Now, there are two ways of doing that:

1) Performing a traditional aggregation to find the users who donated less than the threshold, then filtering these users from the donations dataset, either with `where(not(df("user_id").isin(a_local_list)))` or with a join of type "anti-join".

2) Using window functions to add a new column with the aggregated donations per user, and then using a normal filter such as `where(aggregated_donation < threshold)`

Let's implement both and compare the complexity:

First, perform the traditional aggregation and find the users who donated less than 500 in total:

In [0]:
bad_users = joined_df.groupBy('user_id').agg(fn.round(fn.sum('donation_value'),2).alias('sum')).where(fn.sum('donation_value') < 500)
bad_users.orderBy(desc('sum')).show()

+-------+------+
|user_id|   sum|
+-------+------+
|     87|497.61|
|     73| 455.2|
|     85|437.79|
|     25|413.09|
|     99|370.99|
|     96|354.97|
|     47|351.84|
|     84|329.47|
|     82|291.73|
|     42|215.35|
+-------+------+



You should have found around 10 users. Now, perform an "anti-join" to remove those users from `joined_df`.

**Hint:** The `join` operation accepts a third argument which is the join type. The accepted values are: 'inner', 'outer', 'full', 'fullouter', 'leftouter', 'left', 'rightouter', 'right', 'leftsemi', 'leftanti', 'cross'.

In [0]:
good_donations = joined_df.join(bad_users, 'user_id', 'leftanti')
good_donations.count()

Out[21]: 955

Verify if the count of `good_donations` makes sense by performing a normal join to find the `bad_donations`.

In [0]:
bad_donations = joined_df.join(bad_users, 'user_id', 'leftsemi')
bad_donations.count()

Out[22]: 45

If you done everything right, at this point `good_donations.count()` + `bad_donations.count()` = `joined_df.count()`.

But using the join approach can be very heavy and it requires multipe operations. For this kind of problems, Window Functions are better.

In [0]:
good_donations.count() + bad_donations.count() == joined_df.count()

Out[23]: True

The first step is to create your window specification over `user_id` by using partitionBy

In [0]:
from pyspark.sql import Window

window_spec = Window.partitionBy('user_id')

Then, you can use one of the window functions of the `pyspark.sql.functions` package, appling to the created window_spec by using the `over` method. 

**Hint:** If you are blocked, try looking at the [documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.over) for the `over` method, or searching on StackOverflow.

In [0]:
sum_donation = fn.sum('donation_value').over(window_spec)

donations_with_total = joined_df.withColumn('total_donated_by_user',fn.round(sum_donation,2))
display(donations_with_total.limit(20))

user_id,first_name,last_name,gender,birth_date,country,age,range of age,donation_id,donation_value,date,time,timestamp,total_donated_by_user
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",145,95.88,2017-05-13,5:27:25,2017-05-13 5:27:25,889.16
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",264,150.51,2017-09-21,4:36:37,2017-09-21 4:36:37,889.16
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",317,51.55,2017-10-26,1:55:37,2017-10-26 1:55:37,889.16
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",486,188.37,2017-02-01,20:20:44,2017-02-01 20:20:44,889.16
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",560,121.82,2016-12-03,3:34:03,2016-12-03 3:34:03,889.16
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",561,195.98,2017-09-01,4:04:17,2017-09-01 4:04:17,889.16
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",564,85.05,2017-08-11,0:19:06,2017-08-11 0:19:06,889.16
2,Bruis,Danilov,Male,1966-05-28,Brazil,52.0,"(45,55]",36,197.01,2016-11-28,15:41:01,2016-11-28 15:41:01,1297.12
2,Bruis,Danilov,Male,1966-05-28,Brazil,52.0,"(45,55]",54,118.02,2017-05-22,9:40:27,2017-05-22 9:40:27,1297.12
2,Bruis,Danilov,Male,1966-05-28,Brazil,52.0,"(45,55]",138,173.28,2017-06-29,10:03:02,2017-06-29 10:03:02,1297.12


And now you can just filter on the `total_donated_by_user` column:

In [0]:
good_donations_wf = donations_with_total.where(donations_with_total.total_donated_by_user >= 500)

good_donations_wf.count()

Out[26]: 955

If you done everything right, you should obtain `good_donations_wf.count()` = `good_donations.count()`

In [0]:
good_donations_wf.count() == good_donations.count()

Out[27]: True

Window functions also can be useful to find the "next" and "previous" operations by using `functions.lead` and `functions.lag`. In our example, we can use it to find the date interval between two donations by a specific user.

For this kind of Window function, the window specification must be ordered by the date. So, create a new window specification partitioned by the `user_id` and ordered by the donation timestamp.

In [0]:
ordered_window = Window.partitionBy('user_id').orderBy('timestamp')

Now use `functions.lag().over()` to add a column with the timestamp of the previous donation of the same user. Then, inspect the result to see what it looks like.

In [0]:
new_column = fn.lag('timestamp',1).over(ordered_window)

donations_with_lag = good_donations.withColumn('previous donation time', new_column)
display(donations_with_lag.orderBy("user_id", "timestamp").limit(20))

user_id,first_name,last_name,gender,birth_date,country,age,range of age,donation_id,donation_value,date,time,timestamp,previous donation time
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",560,121.82,2016-12-03,3:34:03,2016-12-03 3:34:03,null
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",486,188.37,2017-02-01,20:20:44,2017-02-01 20:20:44,2016-12-03 3:34:03
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",145,95.88,2017-05-13,5:27:25,2017-05-13 5:27:25,2017-02-01 20:20:44
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",564,85.05,2017-08-11,0:19:06,2017-08-11 0:19:06,2017-05-13 5:27:25
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",561,195.98,2017-09-01,4:04:17,2017-09-01 4:04:17,2017-08-11 0:19:06
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",264,150.51,2017-09-21,4:36:37,2017-09-21 4:36:37,2017-09-01 4:04:17
1,Janifer,Lagadu,Female,1940-08-09,United States,77.0,"(55,~]",317,51.55,2017-10-26,1:55:37,2017-10-26 1:55:37,2017-09-21 4:36:37
2,Bruis,Danilov,Male,1966-05-28,Brazil,52.0,"(45,55]",36,197.01,2016-11-28,15:41:01,2016-11-28 15:41:01,null
2,Bruis,Danilov,Male,1966-05-28,Brazil,52.0,"(45,55]",524,91.76,2017-05-07,14:32:42,2017-05-07 14:32:42,2016-11-28 15:41:01
2,Bruis,Danilov,Male,1966-05-28,Brazil,52.0,"(45,55]",460,65.68,2017-05-11,3:32:57,2017-05-11 3:32:57,2017-05-07 14:32:42


Finally, compute the average time it took for each user between two of their consecutive donations (in days), and print the 5 users with the smallest averages. The result must include at least the users' id, last name and birth date, as well as the computed average.

In [0]:
days_between_donations = donations_with_lag.groupBy('user_id').agg(fn.round(fn.avg(fn.datediff(donations_with_lag.timestamp, donations_with_lag['previous donation time'])),0).alias('average_time_between_donations'))

users_average_between_donations = donations_with_lag.join(days_between_donations,'user_id','inner').select(donations_with_lag.user_id, donations_with_lag.first_name, donations_with_lag.last_name, donations_with_lag.birth_date, 'average_time_between_donations') .distinct().orderBy('average_time_between_donations')
                                              
users_average_between_donations.show(5)

+-------+-----------+---------+----------+------------------------------+
|user_id| first_name|last_name|birth_date|average_time_between_donations|
+-------+-----------+---------+----------+------------------------------+
|     67|     Zollie| Hambatch|1954-04-14|                          14.0|
|     78|Christoforo| Sherston|1965-07-23|                          16.0|
|     91|      Salem|  Kildale|1979-01-25|                          18.0|
|     63|      Tabor|   Bramer|1983-05-29|                          19.0|
|     34| Georgianne| O' Brian|1975-04-12|                          19.0|
+-------+-----------+---------+----------+------------------------------+
only showing top 5 rows



Congratulations, you have finished this notebook!